In [2]:
import pandas as pd
import numpy as np
import re
import MeCab
data = pd.read_excel('idiom_mod_data.xlsx')

In [3]:
# load raw dataset and drop nan
df = data[['어휘','수정']]
df = df.dropna(axis=0) # 애매한 의미 제거 
df.head(30)

,어휘,수정
0,가꾸로 박히다,머리를 땅으로 향한 채 떨어지다.\n
1,가난(이) 들다,가난하게 되다.
2,가난이 파고들다,더욱더 가난해지다.\n
3,가늠(을) 보다,형편이나 시세 따위를 살피다.
4,가늠(을) 잡다,형편이나 시세 따위를 살피다.
5,가늠이 가다,짐작이나 헤아림이 이루어지거나 미치다.\n
6,가닥을 잡다,"분위기, 상황, 생각 따위를 이치나 논리에 따라 바로잡다.\n"
7,가닥이 잡히다,"분위기, 상황, 생각 따위를 이치나 논리에 따라 바로잡게 하다.\n"
8,가도 오도 못하다,한곳에서 자리를 옮기거나 움직일 수 없는 상태가 되다.\n
9,가락(을) 떼다,흥이 나는 일에 첫 동작을 실행하다.


In [4]:
dics = df.values.tolist()

# Preprocessing

In [5]:
# ()처리 포함, 제거 두 예제로 나눔 
idiom_list_pre = []
meaning_list_pre = []
for d in dics:
    idiom = d[0]
    meaning = d[1]
    parentheses = re.findall('\(.*?\)', idiom) # extract 조사 in ()
    
    if len(parentheses) != 0:
        # 1. () 제외
        idiom_list_pre.append(\
                    re.sub('\(.*?\)', '', idiom))
        meaning_list_pre.append(meaning)
#         print(\
#                 re.sub('\(.*?\)', '', idiom))
        # 2. ()안 조사 포함
        idiom_list_pre.append(\
                re.sub('[\(,\)]', '', idiom))
        meaning_list_pre.append(meaning)
#         print(\
#                 re.sub('[\(,\)]', '', idiom))
    else:
        idiom_list_pre.append(idiom)
        meaning_list_pre.append(meaning)
print(len(idiom_list_pre))
print(len(meaning_list_pre))

5371
5371


In [6]:
# 다중 관용구 처리
idiom_list = []
meaning_list = []
for i, m in zip(idiom_list_pre, meaning_list_pre):
    # 의미 빈거 제거 (애매한 의미 등)
    idiom = i.replace('\n','')
    meaning = m.replace('\n','')
    #print(idiom)
    items = re.findall('\[.*?\]', idiom) #extracts string in bracket[] print items
    parentheses = re.findall('\(.*?\)', idiom) # extract 조사 in ()
    
    if len(items) != 0:
        #print('\n\n')
        #print(idiom)
        #print(items)    
        
        # original
        idiom_list.append(\
                re.sub('\[.*?\]', '', idiom))
        meaning_list.append(meaning)
#         print(\
#                 re.sub('\[.*?\]', '', idiom))
        
        # replace pair set
        rep_list = []
        str_list = idiom.split(' ')
        for word in str_list:
            for rep in items:
                if rep in word:
                    rep_list.append([word.replace(rep,''), rep])
        #print(rep_list)

        for pair in rep_list:
            # 유사어 여러개 있을때
            if '/' in pair[1]:
                reps = re.sub('[\[\]]','',pair[1]).split('/')
                #print(reps)
                for r in reps:
                    idiom_list.append(\
                            re.sub('\[.*?\]', '', (idiom.replace(pair[0]+pair[1], r))))
#                     print(\
#                             re.sub('\[.*?\]', '', (idiom.replace(pair[0]+pair[1], r))))
                    meaning_list.append(meaning)
            else:
                idiom_list.append(\
                        re.sub('\[.*?\]', '',\
                            (idiom.replace(pair[0]+pair[1], re.sub('[\[\]]','', pair[1])))))
#                 print(\
#                         re.sub('\[.*?\]', '',\
#                             (idiom.replace(pair[0]+pair[1], re.sub('[\[\]]','', pair[1])))))
                meaning_list.append(meaning)
    
    else:
        idiom_list.append(idiom)
        meaning_list.append(meaning)
        
print(len(idiom_list))
print(len(meaning_list))

5804
5804


In [7]:
print(idiom_list[9])
print(meaning_list[9])

가닥을 잡다
분위기, 상황, 생각 따위를 이치나 논리에 따라 바로잡다.


# tokenizing with spacing token

In [8]:
# 형태소 분석, "/n, . 처리"
# save idiom dictionary to csv file
spacing_tkn = '__'

In [55]:
# 형태소 분석
# " 허리 가 __꼿꼿 하 ㄴ __철수"
mecab = MeCab.Tagger()

def tokenize(text, spacing_tkn='__'):
    # replace white space to spacing token
    text_spacing = text.replace(' ', spacing_tkn)
    # tokenize
    pos = mecab.parse(text_spacing).split('\n')
    tokens = []
    for i, p in enumerate(pos):
        #print(p)
        token = p.split('\t')[0]
        # 연결 어미, 종결 어미 제외
        if "EF" in p or token == "EOS":
            break
        if "EC" in p and len(pos) == i+3:
            break
        if "다" in token and len(pos) == i+3:
            token = token.replace('다','')

        features = p.split(',')
        if features[-1] != '*':
            # 복합 일때
            if '+' in features[-1]: 
                for f in features[-1].split('+'):
                    tokens.append(f.split('/')[0])
            # 단일
            else:
                tokens.append(token)
        else:
            tokens.append(token)
    return tokens


In [56]:
#tokenized = tokenize("분위기, 상황, 생각 따위를 이치나 논리에 따라 바로잡다.")
tokenized = tokenize("꼬리를 물다", spacing_tkn=' ')
print(tokenized)
print(' '.join(tokenized))

['꼬리', '를', '물']
꼬리 를 물


In [58]:
# tokenizing (idiom, meaning)
idiom_df = pd.DataFrame(columns=['idiom', 'meaning'])
for i, m in zip(idiom_list, meaning_list):
    idiom = ' '.join(tokenize(i, spacing_tkn=' '))
    meaning = ' '.join(tokenize(m))
    idiom_df = idiom_df.append(pd.DataFrame([[idiom, meaning]], columns=['idiom', 'meaning']), ignore_index=True)

print(len(idiom_df))
idiom_df.head(10)

5804


,idiom,meaning
0,가꾸로 박히,머리 를 __ 땅 으로 __ 향하 ᆫ __ 채 __ 떨어지
1,가난 들,가난 하 게 __ 되
2,가난 이 들,가난 하 게 __ 되
3,가난 이 파고들,더욱더 __ 가난 하 아 지
4,가늠 보,형편 이나 __ 시세 __ 따위 를 __ 살피
5,가늠 을 보,형편 이나 __ 시세 __ 따위 를 __ 살피
6,가늠 잡,형편 이나 __ 시세 __ 따위 를 __ 살피
7,가늠 을 잡,형편 이나 __ 시세 __ 따위 를 __ 살피
8,가늠 이 가,짐작 이나 __ 헤아리 ᄆ 이 __ 이루어지 거나 __ 미치
9,가닥 을 잡,"분위기 , __ 상황 , __ 생각 __ 따위 를 __ 이치 나 __ 논리 에 __..."


In [59]:
idiom_df.tail(50)

,idiom,meaning
5754,황달 들,누런 빛 을 __ 띠
5755,황달 이 들,누런 빛 을 __ 띠
5756,황모 들,보리 나 __ 밀이 __ 황증 에 __ 걸리 어서 __ 썩 게 __ 되
5757,황모 가 들,보리 나 __ 밀이 __ 황증 에 __ 걸리 어서 __ 썩 게 __ 되
5758,황변 보,아 궁이 의 __ 불 을 __ 때 면서 __ 건조 실 의 __ 담뱃 잎 이 __ 제...
5759,황변 을 보,아 궁이 의 __ 불 을 __ 때 면서 __ 건조 실 의 __ 담뱃 잎 이 __ 제...
5760,황 을 그리,욕 되 리만큼 __ 매우 __ 낭패 를 __ 당하
5761,황천 가,사람 이 __ 죽
5762,황천 으로 가,사람 이 __ 죽
5763,황천 객 이 되,사람 이 __ 죽


In [62]:
# save tokenized idiom, meaning to csv file
idiom_df.to_csv('tokenized_idiom.csv', index=False)

# TEST

In [45]:
def inflect(text, spacing_tkn='__'):
    # replace white space to spacing token
    text_spacing = text.replace(' ', spacing_tkn)
    # tokenize
    pos = mecab.parse(text_spacing).split('\n')
    tokens = []
    for i, p in enumerate(pos):
        #print(p)
        token = p.split('\t')[0]
        
        # 복합 일때
        features = p.split(',')
        if features[-1] != '*' and '+' not in features[-1]:
            if token != features[-1].split('/')[0]:
                print(text_spacing)
                print(p)
                print(token, features[-1].split('/')[0])
    return tokens

In [46]:
etm_list = []
for i, m in zip(idiom_list, meaning_list):
    etm_list.extend(inflect(i, spacing_tkn=' '))

가난 들다
들	VV,*,T,들,Inflect,VV,VV,듣/VV/*
들 듣
가난이 들다
들	VV,*,T,들,Inflect,VV,VV,듣/VV/*
들 듣
가난이 파고들다
파고들	VV,*,T,파고들,Inflect,VV,VV,파고듣/VV/*
파고들 파고듣
가늠이 가다
가	VV,*,F,가,Inflect,VV,VV,갈/VV/*
가 갈
가도 오도 못하다
가	VV,*,F,가,Inflect,VV,VV,갈/VV/*
가 갈
가락 나다
나	VV,*,F,나,Inflect,VV,VV,날/VV/*
나 날
가락을 내다
내	VV,*,F,내,Inflect,VV,VV,나/VV/*
내 나
가면 벗다
가	VV,*,F,가,Inflect,VV,VV,갈/VV/*
가 갈
가면 쓰다
가	VV,*,F,가,Inflect,VV,VV,갈/VV/*
가 갈
가면 쓰다
쓰	VV,*,F,쓰,Inflect,VV,VV,쓸/VV/*
쓰 쓸
가면을 쓰다
쓰	VV,*,F,쓰,Inflect,VV,VV,쓸/VV/*
쓰 쓸
가살 쓰다
쓰	VV,*,F,쓰,Inflect,VV,VV,쓸/VV/*
쓰 쓸
가살을 쓰다
쓰	VV,*,F,쓰,Inflect,VV,VV,쓸/VV/*
쓰 쓸
가슴에 멍이 들다
들	VV,*,T,들,Inflect,VV,VV,듣/VV/*
들 듣
가슴에 피멍 들다
들	VV,*,T,들,Inflect,VV,VV,듣/VV/*
들 듣
가슴에 피멍이 들다
들	VV,*,T,들,Inflect,VV,VV,듣/VV/*
들 듣
가시밭길을 가다
가	VV,*,F,가,Inflect,VV,VV,갈/VV/*
가 갈
간 바람 들다
들	VV,*,T,들,Inflect,VV,VV,듣/VV/*
들 듣
간에 바람 들다
들	VV,*,T,들,Inflect,VV,VV,듣/VV/*
들 듣
간언 들다
들	VV,*,T,들,Inflect,VV,VV,듣/VV/*
들 듣
간언이 들다
들	VV,*,T,들,Inflect,VV,VV,듣/VV/*
들 듣
간에 기별도 안 가다
가	VV,*,F,가,Inflect,VV,VV,갈/VV/*
가 갈
간판을 걸다
걸	VV,*,T,걸,Inflect,

In [187]:
etm_list = list(set(etm_list))
print(etm_list)

['이', 'ᆫ', '아', '게', '의', '을', '라는', 'ㄹ', '은', '다는', 'ㄴ', 'ᆯ', 'ᆫ다는']
